###### Vidhya Analytics Case - BigMart Sales

Import Libraries 

In [1]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from scipy.special import boxcox, inv_boxcox
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor

Read Files 

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv("Submit.csv")

Exploratory Data Analysis 

In [3]:
print(submit.head())

  Item_Identifier Outlet_Identifier
0           FDW58            OUT049
1           FDW14            OUT017
2           NCN55            OUT010
3           FDQ58            OUT017
4           FDY38            OUT027


In [4]:
print(train.shape)
print(test.shape)

(8523, 12)
(5681, 11)


In [5]:
print(train.head(),'\n============================================================')
print(test.head())

  Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0           FDA15         9.30          Low Fat         0.016047   
1           DRC01         5.92          Regular         0.019278   
2           FDN15        17.50          Low Fat         0.016760   
3           FDX07        19.20          Regular         0.000000   
4           NCD19         8.93          Low Fat         0.000000   

               Item_Type  Item_MRP Outlet_Identifier  \
0                  Dairy  249.8092            OUT049   
1            Soft Drinks   48.2692            OUT018   
2                   Meat  141.6180            OUT049   
3  Fruits and Vegetables  182.0950            OUT010   
4              Household   53.8614            OUT013   

   Outlet_Establishment_Year Outlet_Size Outlet_Location_Type  \
0                       1999      Medium               Tier 1   
1                       2009      Medium               Tier 3   
2                       1999      Medium               Tier

In [6]:
train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [7]:
print(train.describe(include = 'object').T)

                     count unique                    top  freq
Item_Identifier       8523   1559                  FDW13    10
Item_Fat_Content      8523      5                Low Fat  5089
Item_Type             8523     16  Fruits and Vegetables  1232
Outlet_Identifier     8523     10                 OUT027   935
Outlet_Size           6113      3                 Medium  2793
Outlet_Location_Type  8523      3                 Tier 3  3350
Outlet_Type           8523      4      Supermarket Type1  5577


Data Engineering

In [8]:
# Replacing values in 'Item_Fat_Content' variable :
train['Item_Fat_Content'] = train['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print(train['Item_Fat_Content'].value_counts())

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64


In [9]:
test['Item_Fat_Content'] = test['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print(test['Item_Fat_Content'].value_counts())

Low Fat    3668
Regular    2013
Name: Item_Fat_Content, dtype: int64


In [10]:
#Get the first two characters of ID:
train['Item_Type_Combined'] = train['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
train['Item_Type_Combined'] = train['Item_Type_Combined'].map({'FD':'Food', 'NC':'Non-Consumable', 'DR':'Drinks'}) 
train['Item_Type_Combined'].value_counts()

Food              6125
Non-Consumable    1599
Drinks             799
Name: Item_Type_Combined, dtype: int64

In [11]:
#Get the first two characters of ID:
test['Item_Type_Combined'] = test['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
test['Item_Type_Combined'] = test['Item_Type_Combined'].map({'FD':'Food', 'NC':'Non-Consumable', 'DR':'Drinks'}) 
test['Item_Type_Combined'].value_counts()

Food              4076
Non-Consumable    1087
Drinks             518
Name: Item_Type_Combined, dtype: int64

In [12]:
# Creating new variable total years from Outlet_Establishment_Year
train['total_years']= 2020 - train['Outlet_Establishment_Year']
test['total_years']= 2020 - train['Outlet_Establishment_Year']

In [13]:
train.drop(['Item_Identifier','Outlet_Establishment_Year'],axis=1,inplace = True)

In [14]:
test.drop(['Item_Identifier','Outlet_Establishment_Year'],axis=1,inplace = True)

In [15]:
print(train.describe())

       Item_Weight  Item_Visibility     Item_MRP  Item_Outlet_Sales  \
count  7060.000000      8523.000000  8523.000000        8523.000000   
mean     12.857645         0.066132   140.992782        2181.288914   
std       4.643456         0.051598    62.275067        1706.499616   
min       4.555000         0.000000    31.290000          33.290000   
25%       8.773750         0.026989    93.826500         834.247400   
50%      12.600000         0.053931   143.012800        1794.331000   
75%      16.850000         0.094585   185.643700        3101.296400   
max      21.350000         0.328391   266.888400       13086.964800   

       total_years  
count  8523.000000  
mean     22.168133  
std       8.371760  
min      11.000000  
25%      16.000000  
50%      21.000000  
75%      33.000000  
max      35.000000  


In [16]:
print(train.describe(include = 'object').T)

                     count unique                    top  freq
Item_Fat_Content      8523      2                Low Fat  5517
Item_Type             8523     16  Fruits and Vegetables  1232
Outlet_Identifier     8523     10                 OUT027   935
Outlet_Size           6113      3                 Medium  2793
Outlet_Location_Type  8523      3                 Tier 3  3350
Outlet_Type           8523      4      Supermarket Type1  5577
Item_Type_Combined    8523      3                   Food  6125


Checking for other data metrics - skewness, kurtosis

In [17]:
print("skewness before transformation \n ",train.skew(axis = 0, skipna = True))
print("Skewness of target variable \n ",train['Item_Outlet_Sales'].skew())
# applying log on target variable     
log_y = np.log(train['Item_Outlet_Sales'])
print("skewness after log \n",log_y.skew())
# applying sqrt on target variable
sqrt_y = np.sqrt(train['Item_Outlet_Sales'])
print("skewness after sqrt \n",sqrt_y.skew())
#applying boxcox on target variable
boxcox_y = stats.boxcox(train['Item_Outlet_Sales'])[0]
print("skewness after boxcox \n ",pd.Series(boxcox_y).skew())
print(train['Item_Outlet_Sales'].head())
print(inv_boxcox(boxcox_y,.347))

skewness before transformation 
  Item_Weight          0.082426
Item_Visibility      1.167091
Item_MRP             0.127202
Item_Outlet_Sales    1.177531
total_years          0.396641
dtype: float64
Skewness of target variable 
  1.1775306028542798
skewness after log 
 -0.887753343209305
skewness after sqrt 
 0.23467599347099247
skewness after boxcox 
  -0.0749780634829317
0    3735.1380
1     443.4228
2    2097.2700
3     732.3800
4     994.7052
Name: Item_Outlet_Sales, dtype: float64
[3704.29144842  441.05298554 2081.6421515  ... 1185.15075816 1832.17223952
  761.02086604]


In [18]:
train.corr()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,total_years
Item_Weight,1.000000,-0.014048,0.027141,0.014123,0.011588
Item_Visibility,-0.014048,1.000000,-0.001315,-0.128625,0.074834
Item_MRP,0.027141,-0.001315,1.000000,0.567574,-0.005020
Item_Outlet_Sales,0.014123,-0.128625,0.567574,1.000000,0.049135
total_years,0.011588,0.074834,-0.005020,0.049135,1.000000


Splitting the train-test file 

In [19]:
y = train['Item_Outlet_Sales']
X= train.drop(['Item_Outlet_Sales'], axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20,random_state=123) 

In [20]:
print(type(X))
print(type(y))

print(type(X_train))
print(type(y_train))

print(type(X_valid))
print(type(y_valid))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


Split Catogorical and Numerical Variables

In [21]:
cat_attr = list(train.select_dtypes("object").columns)
num_attr = list(train.columns.difference(cat_attr))

In [22]:
cat_attr

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type',
 'Item_Type_Combined']

In [23]:
print(num_attr)
num_attr.pop(1)
num_attr

['Item_MRP', 'Item_Outlet_Sales', 'Item_Visibility', 'Item_Weight', 'total_years']


['Item_MRP', 'Item_Visibility', 'Item_Weight', 'total_years']

In [24]:
df_cat_train = X_train[cat_attr]
df_cat_val = X_valid[cat_attr]
df_num_train = X_train[num_attr]
df_num_val = X_valid[num_attr]


In [25]:
test_cat = test[cat_attr]
test_num = test[num_attr]

In [26]:
print(df_cat_train.head(2))
print(df_num_train.head(2))
print(test_num.head(2))
print(test_cat.head(2))

     Item_Fat_Content           Item_Type Outlet_Identifier Outlet_Size  \
6926          Regular         Snack Foods            OUT019       Small   
5168          Low Fat  Health and Hygiene            OUT049      Medium   

     Outlet_Location_Type        Outlet_Type Item_Type_Combined  
6926               Tier 1      Grocery Store               Food  
5168               Tier 1  Supermarket Type1     Non-Consumable  
      Item_MRP  Item_Visibility  Item_Weight  total_years
6926   86.8514         0.216108          NaN           35
5168   33.3874         0.055076         13.5           21
   Item_MRP  Item_Visibility  Item_Weight  total_years
0  107.8622         0.007565        20.75           21
1   87.3198         0.038428         8.30           11
  Item_Fat_Content    Item_Type Outlet_Identifier Outlet_Size  \
0          Low Fat  Snack Foods            OUT049      Medium   
1          Regular        Dairy            OUT017         NaN   

  Outlet_Location_Type        Outlet_Type

In [27]:
print(train.isnull().sum())
print(test.isnull().sum())

Item_Weight             1463
Item_Fat_Content           0
Item_Visibility            0
Item_Type                  0
Item_MRP                   0
Outlet_Identifier          0
Outlet_Size             2410
Outlet_Location_Type       0
Outlet_Type                0
Item_Outlet_Sales          0
Item_Type_Combined         0
total_years                0
dtype: int64
Item_Weight              976
Item_Fat_Content           0
Item_Visibility            0
Item_Type                  0
Item_MRP                   0
Outlet_Identifier          0
Outlet_Size             1606
Outlet_Location_Type       0
Outlet_Type                0
Item_Type_Combined         0
total_years                0
dtype: int64


In [28]:
#out_size = train.groupby(['Item_Outlet_Sales','Outlet_Size']).mean()

In [29]:
train['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [30]:
test['Outlet_Size'].value_counts()

Medium    1862
Small     1592
High       621
Name: Outlet_Size, dtype: int64

Imputing Missing Values using Mode and Mean values

In [31]:
# Impute on train
df_cat_train = df_cat_train.fillna(df_cat_train.mode().iloc[0])

# Impute on test
df_cat_val = df_cat_val.fillna(df_cat_val.mode().iloc[0])

# test
test_cat = test_cat.fillna(test_cat.mode().iloc[0])

In [32]:
# Impute on train
df_num_train = df_num_train.fillna(df_num_train.mean())

#Impute on val
df_num_val = df_num_val.fillna(df_num_val.mean())

# Impute on test
test_num = test_num.fillna(test_num.mean())

Combining Numerical and Categorical variables post imputation

In [33]:
# Combine numeric and categorical in train
X_train = pd.concat([df_num_train, df_cat_train], axis = 1)

# Combine numeric and categorical val
X_valid = pd.concat([df_num_val, df_cat_val], axis = 1)

# Combine numeric and categorical in test
test = pd.concat([test_num, test_cat], axis = 1)

In [34]:
print(X_train.shape)
print(test.shape)

(6818, 11)
(5681, 11)


Converting Categorical to Numerical

In [35]:
## Convert Categorical Columns to Dummies
# Train
X_train = pd.get_dummies(X_train, columns=cat_attr, drop_first=True)

# valid
X_valid = pd.get_dummies(X_valid, columns=cat_attr, drop_first=True)

# test
test = pd.get_dummies(test, columns=cat_attr, drop_first=True)

In [36]:
X_train.columns

Index(['Item_MRP', 'Item_Visibility', 'Item_Weight', 'total_years',
       'Item_Fat_Content_Regular', 'Item_Type_Breads', 'Item_Type_Breakfast',
       'Item_Type_Canned', 'Item_Type_Dairy', 'Item_Type_Frozen Foods',
       'Item_Type_Fruits and Vegetables', 'Item_Type_Hard Drinks',
       'Item_Type_Health and Hygiene', 'Item_Type_Household', 'Item_Type_Meat',
       'Item_Type_Others', 'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods',
       'Outlet_Identifier_OUT013', 'Outlet_Identifier_OUT017',
       'Outlet_Identifier_OUT018', 'Outlet_Identifier_OUT019',
       'Outlet_Identifier_OUT027', 'Outlet_Identifier_OUT035',
       'Outlet_Identifier_OUT045', 'Outlet_Identifier_OUT046',
       'Outlet_Identifier_OUT049', 'Outlet_Size_Medium', 'Outlet_Size_Small',
       'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3',
       'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
       'Outlet_Type_Supermarket

In [37]:
scaler = StandardScaler()

scaler.fit(X_train.loc[:,num_attr])

# scale on train
X_train.loc[:,num_attr] = scaler.transform(X_train.loc[:,num_attr])

# scale on valid
X_valid.loc[:,num_attr] = scaler.transform(X_valid.loc[:,num_attr])

# scale on test
test.loc[:,num_attr] = scaler.transform(test.loc[:,num_attr])

Model Building-1: Support Vector Regression

In [39]:
svr = SVR() 
svr.fit(X_train,y_train)
svr_train_pred = svr.predict(X_train)
svr_valid_pred = svr.predict(X_valid)
svr_test_pred = svr.predict(test)
print(np.sqrt(mean_squared_error(y_train, svr_train_pred)))
#print(MAPE(y_valid, linear_reg_valid_pred))
print(np.sqrt(mean_squared_error(y_valid, svr_valid_pred)))


1663.4388092666932
1636.5855772049254


In [40]:
test_pred_svr = pd.DataFrame(svr_test_pred)
#sub = pd.concat([test_data['id'],test_predict],axis=1)
sub = pd.concat([submit,test_pred_svr],axis = 1)
sub.to_csv('svr_reg.csv',index= False)

# svr with parameter tuning

In [41]:
parameters = {'kernel': ('rbf','poly'), 
              'C':[1.5, 10],'gamma': [1e-7, 1e-4],
              'epsilon':[0.1,0.2,0.5,0.3]}
svr_gs = GridSearchCV(svr, parameters)
svr_gs.fit(X_train,y_train)                                                                                            

GridSearchCV(cv=None, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.5, 10], 'epsilon': [0.1, 0.2, 0.5, 0.3],
                         'gamma': [1e-07, 0.0001], 'kernel': ('rbf', 'poly')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [42]:
best_param = svr_gs.best_params_
print(best_param)

{'C': 10, 'epsilon': 0.2, 'gamma': 0.0001, 'kernel': 'rbf'}


In [43]:
svr_best= SVR(**best_param)
svr_best.fit(X_train,y_train)
svr_best_train_pred = svr_best.predict(X_train)
svr_best_valid_pred = svr_best.predict(X_valid)
svr_best_test_pred = svr_best.predict(test)
print(np.sqrt(mean_squared_error(y_train, svr_best_train_pred)))
#print(MAPE(y_valid, linear_reg_valid_pred))
print(np.sqrt(mean_squared_error(y_valid, svr_best_valid_pred)))

1746.6520874766265
1719.2329504819056


In [44]:
test_pred_svr_best = pd.DataFrame(svr_best_test_pred)

#sub = pd.concat([test_data['id'],test_predict],axis=1)
sub = pd.concat([submit,test_pred_svr_best],axis = 1)
sub.to_csv('svr_best.csv',index= False)

Model Building-2: Tranformed Target Regression

In [45]:
regr_trans = TransformedTargetRegressor(regressor=LinearRegression(),
                                        func=np.log1p,
                                        inverse_func=np.expm1)
regr_trans.fit(X_train, y_train)
regr_trans_train_pred = regr_trans.predict(X_train)
regr_trans_valid_pred = regr_trans.predict(X_valid)
regr_trans_test_pred = regr_trans.predict(test)
print(np.sqrt(mean_squared_error(y_train, regr_trans_train_pred)))
#print(MAPE(y_valid, linear_reg_valid_pred))
print(np.sqrt(mean_squared_error(y_valid, regr_trans_valid_pred)))

1141.704394759487
1132.5105354906877


In [46]:
test_pred = pd.DataFrame(regr_trans_test_pred)

#sub = pd.concat([test_data['id'],test_predict],axis=1)
sub = pd.concat([submit,test_pred],axis = 1)
sub.to_csv('regr_trans.csv',index= False)

In [38]:
boxcox_train = stats.boxcox(y_train)[0]
boxcox_valid = stats.boxcox(y_valid)[0]


In [39]:
from xgboost.sklearn import XGBRegressor

params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf =XGBRegressor(**params)

clf.fit(X_train, boxcox_train)
train_pred_xg = clf.predict(X_train)
val_pred_xg = clf.predict(X_valid)
test_pred_xg = clf.predict(test)
print(np.sqrt(mean_squared_error(boxcox_train, train_pred_xg)))
print(np.sqrt(mean_squared_error(boxcox_valid, val_pred_xg)))

[13:44:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
6.384702521441244
6.339024758938015


In [40]:
# inersing boxcox 
train_pred_xg = inv_boxcox(train_pred_xg,.347)
y_train_inv = inv_boxcox(boxcox_train,.347)
print(np.sqrt(mean_squared_error(y_train_inv, train_pred_xg)))

val_pred_xg = inv_boxcox(val_pred_xg,.347)
y_val_inv = inv_boxcox(boxcox_valid,.347)
print(np.sqrt(mean_squared_error(y_val_inv, val_pred_xg)))

test_pred_xg = inv_boxcox(test_pred_xg,.347)


1083.2663822198476
954.9596250766928


In [41]:
test_pred = pd.DataFrame(test_pred_xg)

#sub = pd.concat([test_data['id'],test_predict],axis=1)
sub = pd.concat([submit,test_pred],axis = 1)
sub.to_csv('xgb_best.csv',index= False)